In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Приведение ряда к стационарному

Для начала рассмотрим временной ряд цен за дюжину яиц. Загрузим данные и отобразим их на графике. Можно сразу заметить наличие понижающегося тренда.

In [ ]:
eggs = pd.read_csv('price-of-a-dozen-eggs.csv', index_col='Year', parse_dates=['Year'])

In [ ]:
eggs.head()

In [ ]:
eggs.plot(figsize=(12,6))
plt.show()

Для проверки стационарности ряда можно воспользоваться критерием Дики-Фуллера, реализация которого находится в модуле `statsmodels.tsa.stattools`.

In [ ]:
from statsmodels.tsa.stattools import adfuller

Данная функция возвращает значение статистики $t$ Дики-Фуллера, p-value, а также значения $t_{крит}$ при заданных уровнях значимости $\alpha$ $0.01, 0.05$ и $0.1$. Если p-value < 0.05, то гипотеза о стационарности ряда не отвергается.

In [ ]:
print("Критерий Дики-Фуллера: p=%f" % adfuller(eggs['Price of a dozen eggs'])[1])

Очевидно, из-за наличия в ряде тренда, ряд не является стационарным.

<div class="alert alert-info">

<h4> Задание 1. Выполнять в отдельном файле</h4>
<p></p>
<ol>
<li>Попробуйте привести исходный ряд к стационарному путем вычитания из него тренда. Для этого можете воспользоваться функцией scipy.linalg.solve. В качестве значений $a$ выступают моменты времени $t=0,1,...,n$, где $n$ $-$ длина ряда, а в качестве $b$ $-$ значения ряда. Выражение $a \times x$ будет описывать тренд.</li>
    <p></p>
<li>С помощью критерия Дики-Фуллера проверьте наличие единичных корней. Выведите график полученного ряда. Является ли полученный ряд стационарным?</li>
    <p></p>
<li>Продифференцируйте исходный ряд. Выведите p-value критерия Дики-Фуллера и постройте график полученного ряда. Какое преобразование является более предпочтительным и почему?</li>
</ol>
</div>

<p></p>
</div>

# 2. Выбор гиперпараметров модели

Построение модели ARIMA рассмотрим на примере данных об объемах пассажирских авиаперевозок.

In [ ]:
passengers = pd.read_csv('passengers.csv', index_col=0, parse_dates=True)

In [ ]:
passengers.plot(figsize=(12,6))
plt.show()

Согласно теореме Вольда стационарный ряд можно описать моделью ARMA. Возникает вопрос: как определить порядок авторегрессии и порядок скользящего среднего? Для начала исходный ряд приводят к стационарному. После чего для определения порядка скользящего среднего прибегают к анализу коррелограммы полученного ряда.

In [ ]:
from scipy.stats import boxcox

In [ ]:
passengers_transformed, lmbda = boxcox(passengers['num_passengers'])
y_transformation = pd.DataFrame(passengers['num_passengers'].values, columns=['initial'])
y_transformation['transformed'] = passengers_transformed
y_transformation['seasonal_diff'] = y_transformation['transformed'].diff(12)
y_transformation['lag1_diff'] = y_transformation['seasonal_diff'].diff(1)

In [ ]:
print("Критерий Дики-Фуллера для исходного ряда: p=%f" % adfuller(y_transformation['initial'])[1])
print("После преобразования Бокса-Кокса: p=%f" % adfuller(y_transformation['transformed'])[1])
print("После сезонного дифференцирования: p=%f" % adfuller(y_transformation['seasonal_diff'].dropna())[1])
print("После дополнительного дифференцирования: p=%f" % adfuller(y_transformation['lag1_diff'].dropna())[1])

Рассмотрим коррелограмму полученного ряда.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)
plot_acf(y_transformation['lag1_diff'].dropna(), lags=48)
plt.xticks(np.arange(0, 50, 2))
plt.show()

В качестве начального приближения гиперпараметра $q$ берется последний значимый лаг автокорреляции. Для примера выше последний значимый лаг равен 1. При лаге равном 2 значение автокорреляции попадает внутрь голубого "коридора", поэтому дальнейшие значения автокорреляции не рассматриваются. Итак, $q_0=1$.

**Замечание по терминологии**: гиперпараметрами модели называют те параметры, которые задаются пользователем. Параметрами же называют подобранные путем минимизации функционала качества моделью коэффициенты. Функционалом качества может выступать, например, среднеквадратичная ошибка.

Т.е. когда речь идет о модели $ARIMA(p,d,q)$, гиперпараметрами являются $p$, $d$ и $q$, которые задаются вручную. Параметрами при этом являются коэффициенты $\alpha$, $\phi$ и $\theta$.

Для выбора начального приближения порядка авторегрессии $p$ прибегают к аналогичному методу, но рассматривают частичную автокорреляцию. Частичная автокорреляция $-$ это автокорреляция после снятия авторегрессии предыдущего порядка. Для определения значения частичной автокорреляции с лагом 2 необходимо построить авторегрессию порядка 1 ($AR(1)$), вычесть эту авторегрессию из ряда и вычислить автокорреляцию на полученных остатках, т.е.:

$$\phi_{hh}=\begin{cases} r(y_{t+1},y_t), & h=1, \\ 
r(y_{t+h}-y_{t+h}^{h-1},y_t-y_t^{h-1}), & h\ge1, \end{cases}$$

где $y_t^{h-1} -$ авторегрессии следующего вида:
$$y_t^{h-1}=\beta_1 y_{t+1}+\beta_2 y_{t+2}+...+\beta_{h-1} y_{t+h-1},$$
$$y_{t+h}^{h-1}=\beta_1 y_{t+h-1}+\beta_2 y_{t+h-2}+...+\beta_{h-1} y_{t+1}.$$

<div class="alert alert-warning">

<h4> Задание 2. Для обсуждения на форуме</h4>
<p></p>
Подумать, почему именно частичная автокорреляция отвечает за порядок авторегрессии, а автокорреляция $-$ за порядок скользящего среднего. Просьба создать обсуждение данного вопроса на de.unecon.
</div>

<div class="alert alert-info">

<h4> Задание 3. Реализация модели AR</h4>
<p></p>
Для полученного стационарного ряда объемов пассажирских авиаперевозок вычислить значение частичной автокорреляции с лагом 3. Для этого необходимо реализовать модель авторегрессии (самостоятельно, без применения средств statsmodels).

<p></p>
</div>

Для построения графика частичной автокорреляции служит функция `plot_pacf`, которая находится в том же модуле `statsmodels.graphics.tsaplots`.

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)
plot_pacf(y_transformation['lag1_diff'].dropna(), lags=48)
plt.xticks(np.arange(0, 50, 2))
plt.show()

Последний значимый лаг также равен 1, соответственно, $p_0=1$. Для выбора оптимальных значений гиперпараметров $p$ и $q$ необходимо построить соответствующие модели авторегрессии и оценить их качество. Поскольку вариантов достаточно много, обычно рассматриваются гиперпараметры в окрестности начального приближения  $p_0$, $q_0$.

In [ ]:
p = range(0, 3)
q = range(0, 3)
d = 1

In [ ]:
from itertools import product

In [ ]:
parameters = product(p, q)
parameters_list = list(parameters)

In [ ]:
parameters_list

Получили 9 возможных наборов гиперпараметров. Гиперпараметры модели нельзя выбирать методом максимального правдоподобия, поскольку с увеличением количества параметров значение функции правдоподобия $L$ растет. 

Поэтому для сравнения различных моделей применяется информационный критерий Акаике:

$$AIC=-2L+2k,$$
где $k$ $-$ число параметров модели. Чем меньше значение данного критерия, тем лучше.

# 3. Построение модели ARIMA

Модель ARIMA можно найти в модуле `statsmodels.tsa.arima_model`. Процесс обучения модели (настройки параметров модели) аналогичен процессу обучения адаптивных методов.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

Обучим модель `ARIMA(1,1,1)`. В качестве первого аргумента функции `ARIMA` необходимо передать временной ряд. Чтобы определить порядки авторегрессии, дифференцирования и скользящего среднего необходимо передать их в виде кортежа в аргумент `order`. В данном случае передадим временной ряд, полученный после сезонного дифференцирования, поскольку обычное дифференцирование функция произведет самостоятельно.

In [ ]:
model = ARIMA(y_transformation['seasonal_diff'].dropna().values, order=(1, 1, 1)).fit()

Значение критерия Акаике:

In [ ]:
model.aic

<div class="alert alert-info">

<h4> Задание 4</h4>
<p></p>
<ol>
<li>Среди ранее полученных наборов гиперпараметров (parameters_list) найдите тот, для которого критерий Акаике минимален.</li>
    <p></p>
<li>С помощью функции predict (именно predict!) получите прогноз на 12 точек вперед.</li>
    <p></p>
<li>Выполните обратные преобразования, чтобы получить прогноз для исходного ряда. Отобразите исходный ряд и прогноз на графике.</li>
</ol>
</div>

<p></p>
</div>